In [72]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [73]:
def canny(img):

    canny = cv.Canny(img, 50, 150)
    return canny

In [74]:
def region_of_interest(img):
    height = img.shape[0]
    poly = np.array([[(200,height), (1200, height), (790,450), (580, 450)]])
    mask = np.zeros_like(img)
    cv.fillPoly(mask, poly, 255)
    return mask

In [75]:
def display_lines(img, lines):
    line_image = np.zeros_like(img)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv.line(line_image, (x1,y1), (x2,y2), (0,255,0), 10)
    return line_image

In [76]:
def average_slope_intercept(lines):
    left_lines    = [] # (slope, intercept)
    left_weights  = [] # (length,)
    right_lines   = [] # (slope, intercept)
    right_weights = [] # (length,)
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x2==x1:
                continue # ignore a vertical line
            slope = (y2-y1)/(x2-x1)
            intercept = y1 - slope*x1
            length = np.sqrt((y2-y1)**2+(x2-x1)**2)
            if slope < 0: # y is reversed in image
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
    
    # add more weight to longer lines    
    left_lane  = np.dot(left_weights,  left_lines) /np.sum(left_weights)  if len(left_weights) >0 else None
    right_lane = np.dot(right_weights, right_lines)/np.sum(right_weights) if len(right_weights)>0 else None
    
    return left_lane, right_lane # (slope, intercept), (slope, intercept)

In [77]:
def make_line_points(y1, y2, line):
    if line is None:
        return None
    
    slope, intercept = line
    print("slope = ", slope)
    print("intercept = ", intercept)
    
    # make sure everything is integer as cv2.line requires it
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    
    return ((x1, y1), (x2, y2))

In [80]:
def lane_lines(image, lines):
    left_lane, right_lane = average_slope_intercept(lines)
    
    y1 = 680 # bottom of the image
    y2 = y1*0.7         # slightly lower than the middle

    left_line  = make_line_points(y1, y2, left_lane)
    right_line = make_line_points(670, y2, right_lane)
    return left_line, right_line

    
def draw_lane_lines(image, lines, color=[0, 255, 0], thickness=10):
    # make a separate image to draw lines and combine with the orignal later
    line_image = np.zeros_like(image)
    for line in lines:
        if line is not None:
            cv.line(line_image, *line,  color, thickness)
    # image1 * α + image2 * β + λ
    # image1 and image2 must be the same shape.
    return cv.addWeighted(image, 1.0, line_image, 0.8, 0.0)

In [81]:
cap = cv.VideoCapture('project_video.mp4')
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    # Display the resulting frame

    
    hls = cv.cvtColor(frame, cv.COLOR_BGR2HLS)    
    low_yellow = np.uint8([ 10,   0, 100])
    up_yellow = np.uint8([ 40, 255, 255])
    mask = cv.inRange(hls, low_yellow, up_yellow)
    gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
    blur = cv.GaussianBlur(gray, (3,3), 0)
    interestt = blur & region_of_interest(blur)
    
    ret, thresh = cv.threshold(interestt, 210, 240, cv.THRESH_BINARY)
    
    new = mask | thresh
    
    interestt = new & region_of_interest(new)

    new_canny = cv.Canny(interestt, 50, 150)
    lines = cv.HoughLinesP(new_canny, 5, np.pi/180, 100, np.array([]), minLineLength=50, maxLineGap=200)
    x = draw_lane_lines(frame, lane_lines(frame, lines))
    cv.imshow('Frame',x)
        
    # Press Q on keyboard to  exit
    if cv.waitKey(25) & 0xFF == ord('q'):
      break

  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv.destroyAllWindows()

slope =  -0.8995102206962828
intercept =  980.1545434025505
slope =  0.514371549935699
intercept =  97.13314253985659
slope =  -0.9118280003157576
intercept =  985.7665508900877
slope =  0.49338054296985007
intercept =  117.17043175911078
slope =  -0.8882135055519451
intercept =  976.4555738958262
slope =  0.5147517773854472
intercept =  99.25033029811415
slope =  -0.8959063120457436
intercept =  980.799927336348
slope =  0.49466594310560175
intercept =  117.12283325016163
slope =  -0.9075745640590673
intercept =  983.6518106284968
slope =  0.5119908844560812
intercept =  102.27897080894792
slope =  -0.8756567772981586
intercept =  972.053528207067
slope =  0.5021597763788523
intercept =  110.03307000205305
slope =  -0.9110860769197386
intercept =  989.415823335463
slope =  0.5067000282833485
intercept =  107.15607845777676
slope =  -0.9127756240335262
intercept =  989.2949610366423
slope =  0.48901098901098894
intercept =  122.64835164835164
slope =  -0.9029424973489577
intercept =  9

slope =  -0.8047756898377894
intercept =  928.002648180537
slope =  0.5091745913767931
intercept =  110.82788093107968
slope =  -0.8278197898753398
intercept =  937.8662052560935
slope =  0.5204408187671422
intercept =  104.48778828943117
slope =  -0.8228680232105204
intercept =  933.4310459964311
slope =  0.521091406632956
intercept =  103.47263444588165
slope =  -0.8091007209995057
intercept =  926.6728245113669
slope =  0.5316770178110366
intercept =  95.4127283051812
slope =  -0.8328741208996499
intercept =  935.0183398191966
slope =  0.5283921638499727
intercept =  96.67513185711672
slope =  -0.8250950816728567
intercept =  929.8712203031711
slope =  0.5372042580544824
intercept =  88.6180617999004
slope =  -0.8237216256840026
intercept =  930.4115789900171
slope =  0.5367980631902822
intercept =  91.18323070522669
slope =  -0.8033170778011034
intercept =  919.1718247354789
slope =  0.5681939331489324
intercept =  63.378568309536135
